## NBA Position Predictor Project

### Created 16 November 2021

## Creating a Model

In [1]:
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from get_stats import read_data_from_files #nbastuffer_dataframe

In [2]:
data = read_data_from_files() #nbastuffer_dataframe(playoffs=False)
data

,RANK,NAME,TEAM,POS,AGE,GP,MPG,USG%,FTA,FT%,...,PPG,RPG,APG,SPG,BPG,TPG,VI,ORTG,DRTG,Year
0,1,Alex Abrines,Okc,G,25.69,31,19.0,12.2,13,0.923,...,5.3,1.5,0.6,0.55,0.19,0.45,3.5,103.1,103.9,2019
1,2,Quincy Acy,Pho,F,28.51,10,12.3,9.2,10,0.700,...,1.7,2.5,0.8,0.10,0.40,0.40,4.9,87.1,98.5,2019
2,3,Jaylen Adams,Atl,G,22.93,34,12.6,13.5,9,0.778,...,3.2,1.8,1.9,0.38,0.15,0.82,7.0,99.5,108.1,2019
3,4,Steven Adams,Okc,C,25.73,80,33.4,16.4,292,0.500,...,13.9,9.5,1.6,1.49,0.96,1.73,7.1,119.9,102.7,2019
4,5,Bam Adebayo,Mia,C-F,21.73,82,23.3,15.8,226,0.735,...,8.9,7.3,2.2,0.88,0.79,1.48,9.0,120.0,97.2,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,599,Devon Dotson,Was,G,23.70,6,8.9,9.8,0,0.000,...,0.5,1.7,1.3,0.80,0.00,0.50,4.6,86.3,98.6,2023
3004,600,Leandro Bolmaro,Uta,F,22.60,14,4.9,15.9,0,0.000,...,0.4,0.5,0.5,0.10,0.10,0.50,3.8,51.0,107.1,2023
3005,602,Sterling Brown,Lal,G-F,28.20,4,6.1,6.9,0,0.000,...,0.0,2.0,0.5,0.80,0.00,0.00,0.0,77.2,92.5,2023
3006,606,Frank Jackson,Uta,G,24.90,1,5.1,24.6,0,0.000,...,0.0,2.0,1.0,0.00,0.00,0.00,0.0,48.0,120.2,2023


In [3]:
stats_df = data
[(i, list(set(stats_df[stats_df['Year'] == i]['POS']))) for i in set(stats_df["Year"])]

[(2019, ['F-C', 'C-F', 'F-G', 'G-F', 'C', 'F', 'G']),
 (2020, [0, 'F-C', 'C-F', 'F-G', 'G-F', 'C', 'F', 'G']),
 (2021, ['F-C', 'C-F', 'F-G', 'G-F', 'C', 'F', 'G']),
 (2022, ['F-C', 'C-F', 'F-G', 'G-F', 'C', 'F', 'G']),
 (2023, ['F-C', 'C-F', 'F-G', 'G-F', 'C', 'F', 'G'])]

In [4]:
stats_df = stats_df[stats_df['POS'] != 0].copy()  # Nicolo Melli (F) - 2019
stats_df = stats_df[stats_df["MPG"] >= 10 ].copy()  # players that played at least 10 minutes
stats_df = stats_df.drop(["RANK", "NAME", "TEAM", "Year"], axis=1)
stats_df

,POS,AGE,GP,MPG,USG%,FTA,FT%,2PA,2P%,3PA,...,TS%,PPG,RPG,APG,SPG,BPG,TPG,VI,ORTG,DRTG
0,G,25.69,31,19.0,12.2,13,0.923,30,0.500,127,...,0.507,5.3,1.5,0.6,0.55,0.19,0.45,3.5,103.1,103.9
1,F,28.51,10,12.3,9.2,10,0.700,3,0.667,15,...,0.379,1.7,2.5,0.8,0.10,0.40,0.40,4.9,87.1,98.5
2,G,22.93,34,12.6,13.5,9,0.778,36,0.361,74,...,0.474,3.2,1.8,1.9,0.38,0.15,0.82,7.0,99.5,108.1
3,C,25.73,80,33.4,16.4,292,0.500,807,0.596,2,...,0.591,13.9,9.5,1.6,1.49,0.96,1.73,7.1,119.9,102.7
4,C-F,21.73,82,23.3,15.8,226,0.735,471,0.588,15,...,0.623,8.9,7.3,2.2,0.88,0.79,1.48,9.0,120.0,97.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,C-F,29.00,14,10.8,11.9,10,0.700,28,0.393,2,...,0.465,2.3,2.6,0.5,0.90,0.60,0.60,5.3,92.6,99.1
2970,G,25.30,5,13.4,10.0,0,0.000,8,0.500,6,...,0.393,2.2,1.8,1.0,0.80,0.60,0.20,4.7,94.4,98.0
2972,G-F,39.20,8,14.1,9.4,3,0.667,6,1.000,9,...,0.521,2.1,2.1,2.4,0.50,0.40,1.10,6.2,102.3,108.3
2987,F,21.50,6,10.2,14.8,2,1.000,8,0.250,10,...,0.238,1.5,2.0,0.7,0.20,0.00,0.30,5.0,64.8,112.2


In [5]:
#plt.hist(stats_df["AGE"])
print(stats_df.groupby('POS')['POS'].count()) #print(list(set(stats_df['POS'])))

POS
C      208
C-F    101
F      722
F-C    178
F-G    101
G      981
G-F    238
Name: POS, dtype: int64


In [6]:
stats_df

,POS,AGE,GP,MPG,USG%,FTA,FT%,2PA,2P%,3PA,...,TS%,PPG,RPG,APG,SPG,BPG,TPG,VI,ORTG,DRTG
0,G,25.69,31,19.0,12.2,13,0.923,30,0.500,127,...,0.507,5.3,1.5,0.6,0.55,0.19,0.45,3.5,103.1,103.9
1,F,28.51,10,12.3,9.2,10,0.700,3,0.667,15,...,0.379,1.7,2.5,0.8,0.10,0.40,0.40,4.9,87.1,98.5
2,G,22.93,34,12.6,13.5,9,0.778,36,0.361,74,...,0.474,3.2,1.8,1.9,0.38,0.15,0.82,7.0,99.5,108.1
3,C,25.73,80,33.4,16.4,292,0.500,807,0.596,2,...,0.591,13.9,9.5,1.6,1.49,0.96,1.73,7.1,119.9,102.7
4,C-F,21.73,82,23.3,15.8,226,0.735,471,0.588,15,...,0.623,8.9,7.3,2.2,0.88,0.79,1.48,9.0,120.0,97.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,C-F,29.00,14,10.8,11.9,10,0.700,28,0.393,2,...,0.465,2.3,2.6,0.5,0.90,0.60,0.60,5.3,92.6,99.1
2970,G,25.30,5,13.4,10.0,0,0.000,8,0.500,6,...,0.393,2.2,1.8,1.0,0.80,0.60,0.20,4.7,94.4,98.0
2972,G-F,39.20,8,14.1,9.4,3,0.667,6,1.000,9,...,0.521,2.1,2.1,2.4,0.50,0.40,1.10,6.2,102.3,108.3
2987,F,21.50,6,10.2,14.8,2,1.000,8,0.250,10,...,0.238,1.5,2.0,0.7,0.20,0.00,0.30,5.0,64.8,112.2


# Neural Network

In [7]:
# s2 is just G, F, and C; s3 is everything else
s2 = stats_df[(stats_df["POS"] =="G") | (stats_df["POS"] =="F") | (stats_df["POS"] =="C")]
s3 = stats_df[~((stats_df["POS"] =="G") | (stats_df["POS"] =="F") | (stats_df["POS"] =="C"))]

In [8]:
s2

,POS,AGE,GP,MPG,USG%,FTA,FT%,2PA,2P%,3PA,...,TS%,PPG,RPG,APG,SPG,BPG,TPG,VI,ORTG,DRTG
0,G,25.69,31,19.0,12.2,13,0.923,30,0.500,127,...,0.507,5.3,1.5,0.6,0.55,0.19,0.45,3.5,103.1,103.9
1,F,28.51,10,12.3,9.2,10,0.700,3,0.667,15,...,0.379,1.7,2.5,0.8,0.10,0.40,0.40,4.9,87.1,98.5
2,G,22.93,34,12.6,13.5,9,0.778,36,0.361,74,...,0.474,3.2,1.8,1.9,0.38,0.15,0.82,7.0,99.5,108.1
3,C,25.73,80,33.4,16.4,292,0.500,807,0.596,2,...,0.591,13.9,9.5,1.6,1.49,0.96,1.73,7.1,119.9,102.7
5,F,22.19,19,10.2,9.9,4,1.000,13,0.385,23,...,0.424,1.7,1.0,0.3,0.05,0.21,0.32,3.1,85.3,115.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,F,20.30,19,11.6,11.0,4,0.750,18,0.500,35,...,0.411,2.4,0.9,0.6,0.40,0.50,0.10,3.8,97.3,113.2
2958,G,25.90,25,10.3,10.9,9,0.667,41,0.488,16,...,0.500,2.4,0.9,1.2,0.40,0.10,0.20,5.3,117.6,114.5
2963,G,24.80,23,12.0,11.6,3,0.667,58,0.431,2,...,0.424,2.3,1.6,1.7,0.30,0.10,0.70,6.1,93.2,114.2
2970,G,25.30,5,13.4,10.0,0,0.000,8,0.500,6,...,0.393,2.2,1.8,1.0,0.80,0.60,0.20,4.7,94.4,98.0


In [9]:
s2["POS"] = LabelEncoder().fit_transform(s2["POS"])

/var/folders/fg/rrlp0jyd0yx58b9sdy3mnbkm0000gn/T/ipykernel_61950/1265152678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s2["POS"] = LabelEncoder().fit_transform(s2["POS"])


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tqdm import trange


X = s2.drop(["POS"], axis = 1)
y = s2["POS"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

batch_size = 256
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train.values), torch.tensor(Y_train.values))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=True)

#X_test = torch.tensor(X_test.values)
#Y_test = torch.tensor(Y_test.values)

test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test.values), torch.tensor(Y_test.values))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=True)

In [11]:
test_dataset

In [12]:
input_size = 21
hidden_size = 10
output_size = 3


class FFNN(nn.Module):

    def __init__(self):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        #print('x', x)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(x, dim=1)
        return x

model = FFNN()
print(model)

FFNN(
  (fc1): Linear(in_features=21, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)


In [13]:
import numpy as np
def train_one_epoch(train_loader, model, device, optimizer, log_interval, epoch):
    model.train()
    losses = []
    counter = []
    
    for i, (img, label) in enumerate(train_loader):
        img, label = img.float().to(device), label.to(device)
        optimizer.zero_grad()
        #print(img.shape, img)
        #print(label.shape, label)
        outputs = model(img)
        #print(outputs.shape, outputs)
        loss = torch.nn.CrossEntropyLoss()(outputs, label)
        loss.backward()
        optimizer.step()
    
        # Record training loss every log_interval and keep counter of total training images seen
        if (i+1) % log_interval == 0:
            losses.append(loss.item())
            counter.append(
                (i * batch_size) + img.size(0) + epoch * len(train_loader.dataset))

    return losses, counter

def test_one_epoch(test_loader, model, device):
    model.eval()
    test_loss = 0
    num_correct = 0
    
    with torch.no_grad():
        for i, (img, label) in enumerate(test_loader):
            output = model(img.float())
            preds = [np.argmax(row) for row in output]
            num_correct += sum([1 for pred, true in zip(preds, label) if pred == true])
            #pred = torch.round(output)  # round probability into binary classification
            #num_correct += pred.eq(label).sum().item()
            test_loss /= len(test_loader)
            
    test_loss /= len(test_loader.dataset)
    return test_loss, num_correct

In [14]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# Hyperparameters
lr = 0.05
max_epochs=20
gamma = 0.95

# Recording data
log_interval = 1

# Instantiate optimizer (model was created in previous cell)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train_losses = []
train_counter = []
test_losses = []
test_correct = []
for epoch in trange(max_epochs, leave=True, desc='Epochs'):
    train_loss, counter = train_one_epoch(train_loader, model, DEVICE, optimizer, log_interval, epoch)
    test_loss, num_correct = test_one_epoch(test_loader, model, DEVICE)

    # Record results
    train_losses.extend(train_loss)
    train_counter.extend(counter)
    test_losses.append(test_loss)
    test_correct.append(num_correct)


Epochs: 100%|██████████| 20/20 [01:03<00:00,  3.16s/it]


In [15]:
print(f"Test accuracy: {test_correct[-1]/len(test_loader.dataset)}")
test_correct

Test accuracy: 0.5300261096605744


[177,
 200,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203,
 203]

In [16]:
#stats_df.describe()
#stats_df.info()

In [17]:
#divide the data into the input 'X' and the labels 'y'
features = list(s2)[1:]
X        = s2[features] #the observations
y        = s2['POS'] #the label

In [18]:
def get_split(X, y):
    """Get the scaled train-test split for the data"""
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=6, stratify = y)
    
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std, X_test_std = sc.transform(X_train), sc.transform(X_test)    
    
    return X_train_std, X_test_std, y_train, y_test

X_train_std, X_test_std, y_train, y_test = get_split(X, y)

## Bagging

In [27]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier

In [28]:
X_train_std, X_test_std, y_train, y_test = get_split(X, y)
bc = BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=10, random_state=0)
bc = bc.fit(X_train_std, y_train)

In [29]:
y_pred = bc.predict(X_test_std)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.56      0.64        62
           1       0.73      0.72      0.73       217
           2       0.84      0.88      0.86       295

    accuracy                           0.79       574
   macro avg       0.77      0.72      0.74       574
weighted avg       0.79      0.79      0.79       574



## Adaboost

In [30]:
from sklearn.ensemble import AdaBoostClassifier

In [31]:
X_train_std, X_test_std, y_train, y_test = get_split(X, y)
abc = AdaBoostClassifier(n_estimators=10)
abc.fit(X_train_std, y_train)

AdaBoostClassifier(n_estimators=10)

In [32]:
y_pred = abc.predict(X_test_std)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.71      0.61        62
           1       0.63      0.54      0.58       217
           2       0.78      0.81      0.79       295

    accuracy                           0.70       574
   macro avg       0.65      0.69      0.66       574
weighted avg       0.70      0.70      0.69       574



## Gradient Tree Boosting

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
X_train_std, X_test_std, y_train, y_test = get_split(X, y)
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gbc.fit(X_train_std, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [35]:
y_pred = gbc.predict(X_test_std)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.58      0.62        62
           1       0.72      0.75      0.73       217
           2       0.87      0.87      0.87       295

    accuracy                           0.79       574
   macro avg       0.75      0.73      0.74       574
weighted avg       0.79      0.79      0.79       574

